# End-to-End Channels

A channel is an open connect between two programs running over a network.

Both of these channels run over the IP protocol which specifies the IP addresses of the machine.

<br>
<img src="images/06-channel.png" width="400">
<br>

UDP (User Datagram Protocol) is our unreliable channel.

TCP (Transmission Control Protocol) is our reliable channel.

# User Datagram Protocol

When using UDP, packets are simply sent towards their destination.

If they don't arrive, no corrective action is taken.

UDP runs over IP, but adds the ability to specify which application the packets should be sent to.

**UDP Header**

<br>
<img src="images/21-udp.png" width="300">
<br>

When an application sends a UDP packet, it prepends the UDP header to the data and sends this to the IP protocol which will prepend its own header and then the packet will be sent over the network.

# Transmission Control Protocol

TCP opens a two-way byte stream channel between processes.



TCP breaks the byte stream into "segments" that will be sent to the destination.

<br>
<img src="images/22-tcp-byte-stream.png" width="400">
<br>

When we "send" data through a TCP channel, it's buffered and broken into segments which are sent as packets over the network to their destination.

TCP guarantees that all bytes will eventually arrive.

TCP does this via the **sliding window algorithm**.

## The Sliding Window Algorithm

TCP achieves reliability using special Acknowledgment packets (ACKs) and timeouts.

### Stop and Wait

The simplest way is to wait for every packet to be ACK'd before sending the next packet.

<br>
<img src="images/23-stop-wait.png" width="300">
<br>

There are four possible scenarios for transmitting a single packet:

<br>
<img src="images/24-stop-wait-cases.png" width="400">
<br>

Stop and wait isn't efficient.

Every single packets requires an entire round trip before the nect packet can be sent because the current packet has to arrive and then so does the ACK.

A crucial property of networks is the amount of data that can be in transit at any given time.

If we have a network with a bandwidth of 1 Mbps and a latency of 10 seconds, then there are 10Mbits "on the wire" at any given time.

This is our bandwidth x latency product and it gives a measure of how many packets can be between the sender and receiver at any given time.

How many packets can you transmit before the first has arrived to its destination?

If a packet is 1KB, then it is a tiny fraction of the data that could be in transit over the network.

We can send many packets before the first ACK could arrive.

## Sliding Window

The sender transmits multiple packets at a time in order to better utilize the network capacity.

If we don't get an ACK for some individual packet, then that packet is resent.

If the entire message is a contiguous set of packets to be sent, there will be some window of those packets which have been sent and are awaiting acknowledgements. As ACK's arrive, that window moves forward.

<br>
<img src="images/25-sliding-window.png" width="400">
<br>

SWS: "Sending window size"

LFS: "Last Frame Sent"

LAR: "Latest ACK Received"

The next packets isn't sent until the packet immediately to the right of LAR is ACK'd.

The window moves foward as packets furthest in the past are ACK'd

<br>
<img src="images/26-sw-timeline.png" width="300">
<br>

<br>
<img src="images/27-tcp.png" width="300">
<br>

SequenceNum and AdvertisedWindow are used for Sliding Window.

AdverstisedWindow is set by the receiver based on the size of its buffer for incoming data.

Acknowledge is field that allows for information to be passed back and forth by the TCP protocol

Flags is field that allows for the type of this TCP packet to be specified:
- ACK
- SYN: is the special packet that is sent to start to establish a TCP channel.
- FIN: is used to tear down a TCP channel
- PUSH: indicates that the receiving process should be notified when this packet arrives
- URG: indicates that this packet contains urgent data



## Setting up a TCP Channel

TO set up a TCP channel, the sender and receiver need to agree on their sequence numbers.

SequenceNumbers refer to byte numbers and indicate the number of the first cyte in this packet.

TCP channels are established using a three-way handshake:



<br>
<img src="images/28-three-way-handshake.png" width="300">
<br>

The purpose is to agree on sequence numbers.

The client and server pick (randomly) their own starting sequence numbers, independent of each other.

The client starts by sending a SYN packet listing its SequenceNumber.

The server responds with a SYN-ACK packet listing its own SequenceNumber and acknowledging the clients.

Finally the client responds with an ACK, acknowledging the servers SequenceNumber.

Why choose the numbers randomly?

It is possible for many incarnations of the same TCP channel to be created one after the other.

If a packet from a previous incarnation arrives VERY LATE, but has a valid sequence number for the current incarnation, it will be incorrectly accepted.

Starting SequenceNumbers are chosen randomly as a guard against this. The probability that two subsequence incarnations will use the same range or anywhere close to it is neglible.